<!-- TABS -->
# Multimodal vector search - Image

<!-- TABS -->
## Connect to SuperDuperDB

:::note
Note that this is only relevant if you are running SuperDuperDB in development mode.
Otherwise refer to "Configuring your production system".
:::

In [ ]:
# <tab: MongoDB>
from superduperdb import superduper

db = superduper('mongodb://localhost:27017/documents')

In [ ]:
# <tab: SQLite>
from superduperdb import superduper
db = superduper('sqlite://my_db.db')

In [ ]:
# <tab: MySQL>
from superduperdb import superduper

user = 'superduper'
password = 'superduper'
port = 3306
host = 'localhost'
database = 'test_db'

db = superduper(f"mysql://{user}:{password}@{host}:{port}/{database}")

In [ ]:
# <tab: Oracle>
from superduperdb import superduper

user = 'sa'
password = 'Superduper#1'
port = 1433
host = 'localhost'

db = superduper(f"mssql://{user}:{password}@{host}:{port}")

In [ ]:
# <tab: PostgreSQL>
!pip install psycopg2
from superduperdb import superduper

user = 'postgres'
password = 'postgres'
port = 5432
host = 'localhost'
database = 'test_db'
db_uri = f"postgres://{user}:{password}@{host}:{port}/{database}"

db = superduper(db_uri, metadata_store=db_uri.replace('postgres://', 'postgresql://'))

In [ ]:
# <tab: Snowflake>
from superduperdb import superduper

user = "superduperuser"
password = "superduperpassword"
account = "XXXX-XXXX"  # ORGANIZATIONID-USERID
database = "FREE_COMPANY_DATASET/PUBLIC"

snowflake_uri = f"snowflake://{user}:{password}@{account}/{database}"

db = superduper(
    snowflake_uri, 
    metadata_store='sqlite:///your_database_name.db',
)

In [ ]:
# <tab: Clickhouse>
from superduperdb import superduper

user = 'default'
password = ''
port = 8123
host = 'localhost'

db = superduper(f"clickhouse://{user}:{password}@{host}:{port}", metadata_store=f'mongomock://meta')

In [ ]:
# <tab: DuckDB>
from superduperdb import superduper

db = superduper('duckdb://mydb.duckdb')

In [ ]:
# <tab: Pandas>
from superduperdb import superduper

db = superduper(['my.csv'], metadata_store=f'mongomock://meta')

In [ ]:
# <tab: MongoMock>
from superduperdb import superduper

db = superduper('mongomock:///test_db')

<!-- TABS -->
## Get useful sample data

In [ ]:
# <tab: Image>
!curl -O https://superduperdb-public-demo.s3.amazonaws.com/images.zip && unzip images.zip
import os
from PIL import Image

data = [f'images/{x}' for x in os.listdir('./images') if x.endswith(".png")][:200]
data = [ Image.open(path) for path in data]

In [ ]:
datas = [{'img': d} for d in data[:100]]

<!-- TABS -->
## Insert simple data

After turning on auto_schema, we can directly insert data, and superduperdb will automatically analyze the data type, and match the construction of the table and datatype.

In [ ]:
from superduperdb import Document

table_or_collection = db['documents']

ids = db.execute(table_or_collection.insert([Document(data) for data in datas]))
select = table_or_collection.select()

<!-- TABS -->
## Build multimodal embedding models

Some embedding models such as [CLIP](https://github.com/openai/CLIP) come in pairs of `model` and `compatible_model`.
Otherwise:

In [ ]:
# <tab: Text-Image>
!pip install git+https://github.com/openai/CLIP.git
import clip
from superduperdb import vector
from superduperdb.ext.torch import TorchModel

# Load the CLIP model and obtain the preprocessing function
model, preprocess = clip.load("RN50", device='cpu')

# Define a vector with shape (1024,)

output_datatpye = vector(shape=(1024,))

# Create a TorchModel for text encoding
compatible_model = TorchModel(
    identifier='clip_text', # Unique identifier for the model
    object=model, # CLIP model
    preprocess=lambda x: clip.tokenize(x)[0],  # Model input preprocessing using CLIP 
    postprocess=lambda x: x.tolist(), # Convert the model output to a list
    datatype=output_datatpye,  # Vector encoder with shape (1024,)
    forward_method='encode_text', # Use the 'encode_text' method for forward pass 
)

# Create a TorchModel for visual encoding
model = TorchModel(
    identifier='clip_image',  # Unique identifier for the model
    object=model.visual,  # Visual part of the CLIP model    
    preprocess=preprocess, # Visual preprocessing using CLIP
    postprocess=lambda x: x.tolist(), # Convert the output to a list 
    datatype=output_datatpye, # Vector encoder with shape (1024,)
)

Because we use multimodal models, we define different keys to specify which model to use for embedding calculations in the vector_index.

In [4]:
indexing_key = 'img' # we use img key for img embedding
compatible_key = 'text' # we use text key for text embedding

## Create vector-index

In [ ]:
vector_index_name = 'my-vector-index'

In [ ]:
# <tab: 2-Modalities>
from superduperdb import VectorIndex, Listener

jobs, _ = db.add(
    VectorIndex(
        vector_index_name,
        indexing_listener=Listener(
            key=indexing_key,      # the `Document` key `model` should ingest to create embedding
            select=select,       # a `Select` query telling which data to search over
            model=model,         # a `_Predictor` how to convert data to embeddings
        ),
        compatible_listener=Listener(
            key=compatible_key,      # the `Document` key `model` should ingest to create embedding
            model=compatible_model,         # a `_Predictor` how to convert data to embeddings
            active=False,
            select=None,
        )
    )
)

In [ ]:
query_table_or_collection = select.table_or_collection

## Perform a vector search

We can perform the vector searches using two types of data:

- Text: By text description, we can find images similar to the text description.
- Image: By using an image, we can find images similar to the provided image.

In [ ]:
# <tab: Text>
item = Document({compatible_key: "Find a black dog"})

In [ ]:
# <tab: Image>
from IPython.display import display
search_image = data[0]
display(search_image)
item = Document({indexing_key: search_image})

Once we have this search target, we can execute a search as follows.

In [ ]:
select = query_table_or_collection.like(item, vector_index=vector_index_name, n=5).select()
results = list(db.execute(select))

## Visualize Results

In [ ]:
from IPython.display import display
for result in results:
    display(result[indexing_key])

## Check the system stays updated

You can add new data; once the data is added, all related models will perform calculations according to the underlying constructed model and listener, simultaneously updating the vector index to ensure that each query uses the latest data.

In [ ]:
new_datas = [{'img': d} for d in data[100:200]]
ids = db.execute(table_or_collection.insert(new_datas))